# Bokeh dashboard for accessibility audit

This notebook creates a Bokeh dashboard with the following common components for a baseline accessibility audit. 

In [ ]:
# Display outputs in the Jupyter Notebook

from bokeh.io import output_notebook

output_notebook()

## Line plot

This section uses data from the [Degrees Bokeh Sampledata](https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#module-bokeh.sampledata.degrees).

### Basic plot

A simple line plot with only default options.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh_sampledata.degrees import data
from bokeh.layouts import Row

source = ColumnDataSource(data=data)

p_line = figure(title="Bachelor’s degrees in Engineering earned by women over the years", 
                x_axis_label="Year", y_axis_label="Number of degrees")
p_line.line(x="Year", y="Engineering", source=source)

show(p_line)

### Multi-line

A plot with two line glyphs differentiated with `line_dash`, hover tooltips, and an interactive "mute" legend.

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh_sampledata.degrees import data

source = ColumnDataSource(data=data)

TOOLTIPS = [
    ("Year", "@Year"),
    ("No.of degrees", "$y"),
     ]

p_mline = figure(
    title="Bachelor’s degrees in Engineering & Business earned by women over the years",
    x_axis_label="Year", 
    y_axis_label="Number of degrees",
    tooltips=TOOLTIPS
)

p_mline.line(x="Year", y="Engineering", source=source, legend_label="Engineering")
p_mline.line(x="Year", y="Business", source=source, legend_label="Business", line_dash="dashed")

p_mline.legend.click_policy="mute"

show(p_mline)

## Bar plot

This section uses data from the [Degrees Bokeh Sampledata](https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#module-bokeh.sampledata.degrees).

A vertical bar plot with:

* tilted y-axis labels
* `Viridis` palette used to colour the bars
* a colorbar to display the colour mapping
* a horizontal line drawn at the "mean" using `Span`
* text and arrow to indicate the line denotes the mean

In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Span, Label, Arrow
from bokeh_sampledata.degrees import data
from bokeh.transform import linear_cmap

df_2010 = data[data["Year"] == 2010]
df_2010 = df_2010.transpose()
df_2010 = df_2010.drop("Year").reset_index()
df_2010.columns = ["field", "degrees"]

source = ColumnDataSource(data=df_2010)
low = int(df_2010.degrees.min())
high = int(df_2010.degrees.max())

cmap = linear_cmap(field_name='degrees', palette="Viridis10", low=low, high=high)

p_bar = figure(
    title="Bachelor's Degrees earned by women in 2010 across fields", 
    x_axis_label="Fields", 
    y_axis_label="Number of degrees",
    x_range=list(df_2010.field),
)

p_bar.xaxis.major_label_orientation = 1

r = p_bar.vbar(x="field", top="degrees", color=cmap, source=source)

color_bar = r.construct_color_bar()
p_bar.add_layout(color_bar, 'right')

mean = int(df_2010.degrees.mean())

span_mean = Span(location=mean, dimension='width')
p_bar.add_layout(span_mean)

text = Label(x=2, y=70, x_units='data', text=r'Mean ($$\bar{x}$$)')
p_bar.add_layout(text)

arw = Arrow(x_start=3, y_start=70, start_units="data", x_end=4.5, y_end=mean, end_units="data")
p_bar.add_layout(arw)

show(p_bar)

### Data table

A tabular display of the raw data used in the above bar plot with the `DataTable` widget.

In [ ]:
from bokeh.io import show
from bokeh.models import ColumnDataSource, DataTable, TableColumn

source = ColumnDataSource(data=df_2010)

columns = [
        TableColumn(field="field", title="Field"),
        TableColumn(field="degrees", title="No.of degrees earned in 2010"),
    ]

data_table = DataTable(source=source, columns=columns)

show(data_table)

## Scatter with legend annotations

This section uses the [Penguins Bokeh Sampledata](https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#module-bokeh.sampledata.penguins).

Two scatter plots in a "tab" layout with:

* multiple glyphs types
* dropdown to filter a specific species
* input that changes the glyph size

In [ ]:
from bokeh.layouts import Column
from bokeh.models import CDSView, GroupFilter, ColumnDataSource, CustomJS, Dropdown, Tabs, TabPanel, NumericInput
from bokeh.transform import factor_mark
from bokeh.plotting import figure, show
from bokeh.sampledata.penguins import data

source = ColumnDataSource(data)
view = CDSView(filter=GroupFilter(column_name="species", group="Adelie"))

SPECIES = sorted(data.species.unique())
MARKERS = ['square', 'circle_x', 'triangle']

p1_scatter = figure(title="Full data set", x_axis_label="Bill length (mm)", y_axis_label="Bill depth (mm)")
p1_scatter.scatter(x="bill_length_mm", y="bill_depth_mm", source=source, marker=factor_mark('species', MARKERS, SPECIES), legend_group="species")

p2_scatter = figure(x_axis_label="Bill length (mm)", y_axis_label="Bill depth (mm)", x_range=p1_scatter.x_range, y_range=p1_scatter.y_range)
r = p2_scatter.scatter(x="bill_length_mm", y="bill_depth_mm", source=source, view=view, marker=factor_mark('species', MARKERS, SPECIES))

menu = [("Adelie", "Adelie"), ("Chinstrap", "Chinstrap"), ("Gentoo", "Gentoo")]
dropdown = Dropdown(label="Select species", menu=menu, align="center")
callback = CustomJS(args=dict(view=view, p2_scatter_title=p2_scatter.title), code="""
    const selected_species = cb_obj.item
    console.log(selected_species)
    view.filter.group = selected_species
    view.emit.change()
""")
dropdown.js_on_click(callback)

numeric_input = NumericInput(value=5, low=1, high=20, title="Enter a number to change the glyph size (between 1-20):", align="center")
numeric_input.js_link('value', r.glyph, 'size')

p_filtered_scatter = Tabs(tabs=[
    TabPanel(child=p1_scatter, title="All species"),
    TabPanel(child=Column(dropdown, numeric_input, p2_scatter), title="Selected species"),
])

show(p_filtered_scatter)

## Final Dashboard

A dashboard created with `layout` that combines all the above interactive plots in a single view to export.

In [ ]:
from bokeh.layouts import layout
from bokeh.plotting import show, output_file
from bokeh.models import TabPanel, Tabs, Div

div1 = Div(text=
           """
           <h1> Bokeh dashboard for accessibility audit</h1>
           <h2>Line and bar plots</h2>
           <p> This section uses data from the <a href="https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#module-bokeh.sampledata.degrees">Degrees Bokeh Sampledata</a>.</p>
           <br>
           <p>
           Left-to-right, top-to-bottom:
           <ol>
               <li>Basic line plot with default options.</li>
               <li>A plot with two lines, hover tooltip interactions, and an interactive legend that can be "muted" by clicking on the legend labels.</li>
               <li>Vertical bar plot that uses the <a href="https://docs.bokeh.org/en/latest/docs/reference/palettes.html#matplotlib-palettes">Virdis built-in color palette</a>. It also uses the span, text (including mathematical notation), and arrow annotations to show the mean value.</li>
               <li>The raw data used to create the bar plot displayed with a <a href="https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html#datatable">DataTable widget</a>.</li>
           </ol>
           </p>
           """
)

div2 = Div(text=
           """
           <h2>Scatter plot</h2>
           """
)

div3 = Div(text=
                 """
                 <p> This section uses the <a href="https://docs.bokeh.org/en/latest/docs/reference/sampledata.html#module-bokeh.sampledata.penguins">Penguins Bokeh Sampledata</a>.</p>
                 <p>
                 Two scatter plots displayed in a "tab" layout with:
                   <ul>
                       <li>Multiple glyphs types</li>
                       <li>Dropdown widget to filter for a specific species</li>
                       <li> Numeric input widget that changes the glyph size</li>
                   </ul>
                </p>
                """
)

dashboard = layout([
    [div1],
    [p_line, p_mline],
    [p_bar, data_table],
    [div2],
    [div3, p_filtered_scatter],
])

dashboard.sizing_mode = "stretch_width"

show(dashboard)

## Output to HTML file

Export the dashabord as an HTML file and save it at the root of the respository.

In [ ]:
from bokeh.io import output_file, save

output_file(filename="../index.html", title="Bokeh Dashboard for Accessibility Audit")

save(dashboard)